In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
# Path to the data txt file on disk.
data_path   = 'data/SVID.csv'

# Parameters
batch_size  = 64  # Batch size for training.
epochs      = 20  # Number of epochs to train for.
latent_dim  = 256  # Latent dimensionality of the encoding space.
num_samples = 10  # Number of samples to train on.

In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(',')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.0
    decoder_target_data[i, t:, target_token_index[' ']] = 1.0

Number of samples: 10
Number of unique input tokens: 10
Number of unique output tokens: 9
Max sequence length for inputs: 15
Max sequence length for outputs: 12


In [7]:
input_token_index

{' ': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 'A': 4,
 'B': 5,
 'C': 6,
 'E': 7,
 'H': 8,
 '_': 9}

In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8 samples, validate on 2 samples
Epoch 1/20
8/8 [==============================] - 1s 90ms/step - loss: 2.2019 - accuracy: 0.0833 - val_loss: 2.1061 - val_accuracy: 0.2500
Epoch 2/20
8/8 [==============================] - 0s 3ms/step - loss: 2.1062 - accuracy: 0.2500 - val_loss: 1.9137 - val_accuracy: 0.2500
Epoch 3/20
8/8 [==============================] - 0s 3ms/step - loss: 1.9175 - accuracy: 0.2500 - val_loss: 2.0162 - val_accuracy: 0.2500
Epoch 4/20
8/8 [==============================] - 0s 3ms/step - loss: 2.0354 - accuracy: 0.2500 - val_loss: 1.8200 - val_accuracy: 0.2500
Epoch 5/20
8/8 [==============================] - 0s 3ms/step - loss: 1.8213 - accuracy: 0.2500 - val_loss: 1.5224 - val_accuracy: 0.4167
Epoch 6/20
8/8 [==============================] - 0s 3ms/step - loss: 1.5402 - accuracy: 0.4167 - val_loss: 1.3706 - val_accuracy: 0.3333
Epoch 7/20
8/8 [==============================] - 0s 3ms/step - loss: 1.3982 - accuracy: 0.3333 - val_loss: 2.7963 - val_accuracy

In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ABC CH1 ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH1_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH2_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH3_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CHA_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CHB_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH_1_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH_2_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH_3_ABCHHE
Decoded sentence: ABCCCHHH

-
Input sentence: ABC_CH_A_ABCHHE
Decoded sentence: ABCCCHHH



ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[[1., 0., 0., 0., 0., 0., 0., 0., 0.]]])]...